<a href="https://colab.research.google.com/github/paucimi/frigo-vision-recommender/blob/main/recuperar_y_taguear_imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Instalación de la herramienta de descarga
!pip install fiftyone


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.5/934.5 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 12.5 MB/s eta 0:

In [14]:
#esta parte hay que cambiarla para que lo tome de los parámetros de entrada de la web
# montar el entorno de drive - esto es para que me encuentren los ficheros,

from google.colab import drive
drive.mount('/content/drive')

# incorporar el nombre del proyecto
ruta_proyecto = '/content/drive/MyDrive/'
os.chdir(ruta_proyecto)
base_dir = "./fridge"

# si queremos usar un bucket en lugar de drive, usamos toda esta parte, $GOOGLE_CLOUD_PROJECT con tu ID de proyecto.
GOOGLE_CLOUD_PROJECT ="id de tu proyecto"
base_dir =gs://$GOOGLE_CLOUD_PROJECT-fridge/
gsutil mb -p $GOOGLE_CLOUD_PROJECT -c standard -l us-central1 base_dir




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:

# Descargamos imágenes y etiquetas automáticamente
import fiftyone as fo
import fiftyone.zoo as foz
import os
import csv
import shutil

# 1. Definición de la lista y rutas
lista_alimentos = ["Milk", "Onion", "Orange", "Apple"]
csv_path = os.path.join(base_dir, "data.csv")

# Asegurar que la carpeta base existe
os.makedirs(base_dir, exist_ok=True)


# Preparar el archivo CSV (escribimos la cabecera)
with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["ruta_imagen", "alimento"])

print(f"Iniciando descarga y tagueado en {base_dir}...")

# 2. Bucle recursivo/iterativo por cada alimento
for alimento in lista_alimentos:
    print(f"\n--- Procesando: {alimento} ---")

    # Crear carpeta específica para el alimento
    alimento_dir = os.path.join(base_dir, alimento)
    os.makedirs(alimento_dir, exist_ok=True)

    try:
        # Descarga del dataset para el alimento específico
        dataset = foz.load_zoo_dataset(
            "open-images-v7",
            split="validation",
            classes=[alimento],
            max_samples=10, # Ajusta según necesites
            seed=51,
            drop_existing_dataset=True # Limpia memoria de descargas previas
        )

        # Identificar campo de etiquetas (detections o ground_truth)
        label_field = "detections" if "detections" in dataset.get_field_schema() else "ground_truth"

        # Exportar imágenes a la subcarpeta del alimento
        # Usamos el tipo ImageDirectory porque solo queremos las fotos organizadas
        dataset.export(
            export_dir=alimento_dir,
            dataset_type=fo.types.ImageDirectory,
            label_field=label_field,
        )

        # 3. Generar líneas en el CSV para las imágenes descargadas
        with open(csv_path, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for filename in os.listdir(alimento_dir):
                if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                    ruta_completa = os.path.abspath(os.path.join(alimento_dir, filename))
                    writer.writerow([ruta_completa, alimento])

        print(f"✅ {alimento} completado y registrado en CSV.")

    except Exception as e:
        print(f"❌ Error procesando {alimento}: {e}")

print(f"\n🚀 Proceso finalizado. Archivo creado en: {csv_path}")

Iniciando descarga y tagueado en ./fridge...

--- Procesando: Milk ---


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/open-images-v7/validation' if necessary


Necessary images already downloaded


INFO:fiftyone.utils.openimages:Necessary images already downloaded


Existing download of split 'validation' is sufficient


...

Detected an unlabeled image exporter and a label field 'detections' of type <class 'fiftyone.core.labels.Detections'>. Exporting image patches...


INFO:fiftyone.utils.data.exporters:Detected an unlabeled image exporter and a label field 'detections' of type <class 'fiftyone.core.labels.Detections'>. Exporting image patches...


 100% |███████████████████| 49/49 [1.0s elapsed, 0s remaining, 47.5 samples/s]         


INFO:eta.core.utils: 100% |███████████████████| 49/49 [1.0s elapsed, 0s remaining, 47.5 samples/s]         


✅ Apple completado y registrado en CSV.

🚀 Proceso finalizado. Archivo creado en: ./fridge/data.csv


In [11]:
from google.colab import drive
drive.mount('/content/drive')

./fridge:
Apple  data.csv  Milk  Onion  Orange

./fridge/Apple:
000001.jpg  000010.jpg	000019.jpg  000028.jpg	000037.jpg  000046.jpg
000002.jpg  000011.jpg	000020.jpg  000029.jpg	000038.jpg  000047.jpg
000003.jpg  000012.jpg	000021.jpg  000030.jpg	000039.jpg  000048.jpg
000004.jpg  000013.jpg	000022.jpg  000031.jpg	000040.jpg  000049.jpg
000005.jpg  000014.jpg	000023.jpg  000032.jpg	000041.jpg
000006.jpg  000015.jpg	000024.jpg  000033.jpg	000042.jpg
000007.jpg  000016.jpg	000025.jpg  000034.jpg	000043.jpg
000008.jpg  000017.jpg	000026.jpg  000035.jpg	000044.jpg
000009.jpg  000018.jpg	000027.jpg  000036.jpg	000045.jpg

./fridge/Milk:
000001.jpg  000008.jpg	000015.jpg  000022.jpg	000029.jpg  000036.jpg
000002.jpg  000009.jpg	000016.jpg  000023.jpg	000030.jpg  000037.jpg
000003.jpg  000010.jpg	000017.jpg  000024.jpg	000031.jpg  000038.jpg
000004.jpg  000011.jpg	000018.jpg  000025.jpg	000032.jpg  000039.jpg
000005.jpg  000012.jpg	000019.jpg  000026.jpg	000033.jpg
000006.jpg  000013.jpg	000